<a href="https://colab.research.google.com/github/yaobviously/NBADFS/blob/main/preprocessing_odds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [48]:
nba_odds_19 = '/content/drive/MyDrive/NBA Player Box Scores/nba_odds_2018-19.csv'
nba_odds_20 = '/content/drive/MyDrive/NBA Player Box Scores/nba_odds_2019-20.csv'
nba_odds_21 = '/content/drive/MyDrive/NBA Player Box Scores/nba_odds_2020_21.csv'

In [118]:
def process_csv(csv=nba_odds_21, year=2020):

    odds_cols = ['Date', 'VH', 'Team', 'Final', 'Close', 'ML']

    odds_team_map = {'GoldenState' : 'GSW', 'Brooklyn' : 'BKN', 'LAClippers' : 'LAC',
                    'LALakers':'LAL', 'Charlotte':'CHA', 'Cleveland':'CLE', 
                    'NewYork':'NYK', 'Indiana':'IND', 'Miami':'MIA', 'Orlando':'ORL',
                    'Washington':'WAS', 'Philadelphia':'PHI', 'Milwaukee':'MIL', 
                    'Boston':'BOS', 'NewOrleans':'NOP', 'Toronto':'TOR',
                    'Atlanta':'ATL', 'Chicago':'CHI', 'SanAntonio':'SAS',
                    'Memphis':'MEM', 'Detroit':'DET', 'Minnesota':'MIN',
                    'Sacramento':'SAC', 'Denver':'DEN', 'Utah':'UTA', 
                    'Portland':'POR', 'Dallas':'DAL', 'Phoenix':'PHO',
                    'OklahomaCity':'OKC', 'Houston':'HOU'}

    df = pd.read_csv(csv, usecols=odds_cols, na_values=0)
    df.columns = df.columns.str.lower()
    df = df[df['vh'] != 'N'].copy()

    game_num = []

    for i in range(len(df)//2):
      game_num.append(i)
      game_num.append(i)

    df['game_num'] = game_num
    df['home'] = np.where(df['vh'] == 'H', 1, 0)
    df.drop(columns='vh', inplace=True)

    df['close'] = df['close'].replace('pk', 0).replace('PK', 0).astype(float)

    total_spread = pd.pivot_table(df, values='close', index='game_num', columns='home')
    total_spread.rename(columns={0:'total_line', 1:'spread_line_home_fave'}, inplace=True)

    # swapping the column values where the spread and total are in the wrong col
    condition = (total_spread['spread_line_home_fave'] > 100)

    total_spread.loc[condition, ['total_line', 'spread_line_home_fave']] = (
        total_spread.loc[condition, ['spread_line_home_fave', 'total_line']].values
    )

    # getting actual spread using a pivot table and joining it to the df
    spread_df = pd.pivot_table(df, values='final', columns='home', index='game_num')
    spread_df['actual_spread'] = spread_df[0].sub(spread_df[1])
    
    df = df.join(spread_df[['actual_spread']], how='left', on='game_num')

    # joining the spread df
    df = df.join(total_spread, how='left', on='game_num')
    df['spread'] = np.where(df['home'] == 1, df['spread_line_home_fave'] * -1, df['spread_line_home_fave'])
    df.drop(columns=['spread_line_home_fave', 'close'], inplace=True)
    df['actual_total'] = df.groupby('game_num')['final'].transform(lambda x: x.sum())

    # converting the numeric date indicator to the date format of the other dataframes
    df['day'] = [str(x)[-2:] for x in df['date']]
    df['day'] = [x[1] if x[0] == '0' else x for x in df['day']]
    df['month'] = [int(str(x)[:-2]) for x in df['date']]
    df['year'] = np.where(df['month'] >= 10, year, year+1)
    df['game_date'] = [str(x) +'-' + str(y) + '-' + str(z) for x, y, z in zip(df['year'], df['month'], df['day'])]
    df['game_date'] = pd.to_datetime(df['game_date'])

    # mapping team names to standard team abbreviations using the dict defined above
    df['team'] = df['team'].map(odds_team_map).fillna(df['team'])

    df.drop(columns=['date', 'game_num', 'day', 'month', 'year'], inplace=True)

    return df

In [120]:
df1 = process_csv(nba_odds_21, 2020)
df2 = process_csv(nba_odds_20, 2019)
df3 = process_csv(nba_odds_19, 2018)

odds_df = pd.concat([df3, df2, df1]).sort_values(by='game_date').reset_index(drop=True)

In [123]:
odds_df.to_csv('/content/drive/MyDrive/NBA Player Box Scores/processed_nba_odds_2018_to_2021_no_neutrals.csv', index=False)